In [1]:
# !python -m pip install tushare pyecharts ipywidgets > pip_install.log

In [2]:
import re
import base64
import tushare as ts
import ipywidgets as widgets
import pyecharts.options as opts
from pyecharts.charts import Candlestick

# 用tushare获取股票代码， 然后用pyecharts生成一个k线的html
def get_stock_plot_html(code):
    df = ts.get_k_data(code)
    x_data = [str(x) for x in df['date']]
    y_data = [
        [row['open'], row['close'], row['low'], row['high']]
        for _, row in df.iterrows()
    ]
    c = (
        Candlestick(init_opts=opts.InitOpts(width="600px", height="400px"))
        .add_xaxis(xaxis_data=x_data)
        .add_yaxis(series_name="", y_axis=y_data)
        .set_series_opts()
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(
                splitline_opts=opts.SplitLineOpts(
                    is_show=True, linestyle_opts=opts.LineStyleOpts(width=1)
                )
            )
        )
    )
    out = c.render()
    with open(out) as fp:
        return df, fp.read()

input_text = widgets.Text(description='输入框')
submit_button = widgets.Button(description='提交')
html = widgets.HTML(value='')

def submit(*ignore):
    code = input_text.value
    if re.match(r'^\d{6}$', code):
        # 把pyecharts生成的html转换为base64编码嵌入到结果
        df, html_src = get_stock_plot_html(code)
        html_src_b64 = base64.b64encode(html_src.encode('utf-8')).decode('utf-8')
        html.value = (
            f'<iframe src="data:text/html;base64,{html_src_b64}" '
            ' frameborder="0" style="width: 620px; height: 420px;"></iframe>'
            '<br />' + df.tail().to_html()
        )
    else:
        html.value = f'错误的代码"{code}"，不是6个数字'
    
submit_button.on_click(submit)

widgets.VBox([
    widgets.HBox([input_text, submit_button]),
    html
])